In [1]:
import pandas as pd
import os
import numpy as np
import re

# How datasets are joined

### Movie dataset and Character dataset
We join the two datasets on the `freebase_movie_id`.

### Character dataset and Oscar dataset
Oscar dataset does not have `freebase_movie_id` or `freebase_actor_id`. We instead use `parsed_actor_name` and `movie_identifier`. `parsed_actor_name` will be unique for each movie as we drop actors if they share `parsed_actor_name` from playing another character in the same movie. `movie_identifier` is a combination of `parsed_movie_name` and `release_year`. This is unique as we drop movies that share `movie_identifier`.

### Resulting dataset from previous steps and IMDb dataset
We join these datasets using a combination of `parsed_movie_name` and `release_year` as primary key. 

# Preparing Movie dataset

In [2]:
print("########## Data pipeline ##########")
print("\nPreparing CMU data")

########## Data pipeline ##########

Preparing CMU data


In [3]:
movie_df = pd.read_csv('data/movie.metadata.tsv', sep='\t', names=['wiki_movie_id', 'freebase_movie_id', 'title', 'release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genres'], index_col='freebase_movie_id')
movie_df = movie_df.reset_index()

In [4]:
# This method removes trailing and leading whitespaces, converts names entirely to lowercase (to ensure case differences in names from different datasets don't affect later joins (with IMDb, for example). 
# We also remove any characters that are not [a-zA-Z0-9 -] (alphanumeric, whitespace and '-') 
def parse_string(s):
    try:
        s = s.lower().strip()
        s  = re.sub('[^a-zA-Z0-9 -]', '', s)
        if len(s) == 0:
            return None
        return s
    except:
        return None 

In [5]:
movie_df['parsed_movie_name'] = movie_df['title'].apply(parse_string)
movie_df['release_year'] = movie_df['release_date'].apply(lambda x: pd.to_datetime(x, format='mixed', errors='coerce').year)

In [6]:
movie_df['release_year'] = movie_df['release_year'].fillna(0).astype(int)

In [7]:
#Dropped movies without parsed_movie_name
movie_df = movie_df[movie_df['parsed_movie_name'].notna()]
#Create identifier
movie_df['movie_identifier'] = movie_df.apply(lambda x: x['parsed_movie_name'] + '_' + str(x['release_year']), axis=1)
movie_df = movie_df.drop(['parsed_movie_name'], axis=1)

In [8]:
def extract_column(s):
    s = str(s)
    return re.findall(r'\"([^\/:][\w\s]+)"', s)

In [9]:
# Extract data from columns
movie_df['countries'] = movie_df['countries'].apply(extract_column)
movie_df['languages'] = movie_df['languages'].apply(extract_column)
movie_df['genres'] = movie_df['genres'].apply(extract_column)

# If movie has no language it is assumed to be in english.
movie_df['languages'] = movie_df.apply(lambda x: x['languages'] if len(x['languages']) else ['English Language'], axis=1)

In [10]:
# Remove movies with duplicate movie_identifiers
temp = movie_df.groupby('movie_identifier').agg(count = ('movie_identifier', 'size'))
print(temp[temp['count'] > 1]['count'].sum(), 'movies shared both name and release year, dropping')
movie_df = movie_df.merge(temp[temp['count'] == 1], on='movie_identifier', how='inner')

movie_df = movie_df.drop('count', axis=1)

379 movies shared both name and release year, dropping


# Preparing Character dataset

In [11]:
# Read character
col_names = ["wiki_movie_id", "freebase_movie_id", "movie_release_date", "character_name", "actor_date_of_birth", "Actor gender", "Actor height (in meters)", "Actor ethnicity (Freebase ID)", "Actor name", "Actor age at movie release", "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]
character_df = pd.read_csv('data/character.metadata.tsv', sep='\t', names=col_names, index_col="Freebase character ID")

In [12]:
# Remove characters without actors
character_df = character_df[character_df['Freebase actor ID'].notna()]

# Allow only one character per actor per movie.
# Motivation: When we look at if the actor won an Oscar we don't care about what role it won it as. 
character_df = character_df.groupby(['freebase_movie_id', 'Freebase actor ID']).first().reset_index()

character_df['parsed_actor_name'] = character_df['Actor name'].apply(parse_string)

# How many cases where the actors of the same name stars in the same movie or
temp = character_df.groupby(['freebase_movie_id', 'parsed_actor_name']).agg(count = ('parsed_actor_name', 'size')).reset_index()
movies_with_actors_of_same_name_df = temp[temp['count'] > 1]
print(movies_with_actors_of_same_name_df.shape[0], 'movies had actors with the same name, dropping')

# Only 314 actors had acted multiple times in the same movie we decide to drop the cases.
character_df = character_df.merge(temp, on=['freebase_movie_id', 'parsed_actor_name'], how='inner')
dropped_character_df = character_df[character_df['count'] == 1]

temp = dropped_character_df.groupby(['freebase_movie_id', 'parsed_actor_name']).agg(count = ('parsed_actor_name', 'size')).reset_index()

assert temp[temp['count'] > 1].empty

character_df = dropped_character_df
character_df = character_df.drop('count', axis=1)

314 movies had actors with the same name, dropping


# Merge Movie dataset on character dataset

In [13]:
movie_character_df = movie_df.merge(character_df, on='freebase_movie_id', how='inner')
movie_character_df = movie_character_df[[
    'title',
    'release_date',
    'box_office_revenue',
    'runtime',
    'languages',
    'countries',
    'genres',
    'movie_identifier',
    'Freebase actor ID',
    'Actor gender',
    'Actor height (in meters)',
    'Actor ethnicity (Freebase ID)',
    'Actor name',
    'Actor age at movie release',
    'parsed_actor_name'
]]
movie_character_df['actor_identifier'] = movie_character_df['Freebase actor ID']
movie_character_df = movie_character_df.drop('Freebase actor ID', axis=1)

In [14]:
# Create identifier used for joining on Oscar dataset
movie_character_df['identifier'] = movie_character_df.apply(lambda x: x['movie_identifier'] + '_' + x['parsed_actor_name'], axis=1)
assert movie_character_df['identifier'].is_unique

# Preparing Oscar dataset

In [15]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')

# Removes weird Oscar nomination
oscar_df = oscar_df[oscar_df['film'].notna() & oscar_df['name'].notna()]

In [16]:
# Filtering relevant Oscar categories
# We only consider nominations for actors and actresses. Meaning we disregard nominations for directors, animation, costumes, etc.
oscar_df = oscar_df[oscar_df['category'].str.contains('ACTOR') | oscar_df['category'].str.contains('ACTRESS')]

In [17]:
# Create identifier 
oscar_df['parsed_movie_name'] = oscar_df['film'].apply(parse_string)
oscar_df['parsed_actor_name'] = oscar_df['name'].apply(parse_string)
oscar_df['identifier'] = oscar_df.apply(lambda x: x['parsed_movie_name'] + '_' + str(x['year_film']) + '_' + x['parsed_actor_name'], axis=1)

In [18]:
# If rows share identifier it must mean the same actor got nominated for the 
# same film in multiple categories. We only count one.
oscar_df = oscar_df.groupby(['identifier']).first().reset_index()
assert oscar_df['identifier'].is_unique

In [19]:
# Select relevant columns
oscar_df = oscar_df[[
    'identifier',
    'category',
    'winner'
]]

# Merge movie_character dataset to Oscar
The join is identifier which consists of parsed_movie_name, release_year and parsed_actor_name

In [20]:
movie_character_oscar_df = movie_character_df.merge(oscar_df, how='left', on='identifier')

In [21]:
# Create column to determine if actor was nominated for Oscar 
movie_character_oscar_df['oscar_nominated'] = movie_character_oscar_df['category'].notna()

## Stats about dataset

In [22]:
print("\nMerging Oscar dataset, after merge:")
nominated_df = movie_character_oscar_df[movie_character_oscar_df['oscar_nominated'] == True]
print('Number of different Oscar nominated movies in dataset:', 
      nominated_df['movie_identifier'].unique().shape[0],
      "in total", movie_character_oscar_df['movie_identifier'].unique().shape[0], "different movies")
print('Number of different Oscar nominated actors in dataset:', 
      nominated_df['actor_identifier'].unique().shape[0],
      'in total',movie_character_oscar_df['actor_identifier'].unique().shape[0],"different actors")
print('Number of Oscar nominated rows:', nominated_df.shape[0])


Merging Oscar dataset, after merge:
Number of different Oscar nominated movies in dataset: 952 in total 63968 different movies
Number of different Oscar nominated actors in dataset: 801 in total 134907 different actors
Number of Oscar nominated rows: 1443


# Prepare IMDB rating dataset 

In [23]:
#Read data
titlebasics_df = pd.read_csv('data/title.basics.tsv', sep='\t', quoting=3)
titleratings_df = pd.read_csv('data/title.ratings.tsv', sep='\t')

In [24]:
#Merge dataframes containing ratings and title names
name_rating_df = titlebasics_df.merge(titleratings_df, how='inner', on=['tconst', 'tconst'])
#Consider only movies
name_rating_df = name_rating_df[(name_rating_df["titleType"] == 'movie')]
#Get relevant columns
name_rating_df = name_rating_df[["primaryTitle", "startYear", "averageRating", "numVotes"]]

In [25]:
#Change the year data in IMDB dataset to float for merging to work
#\\N is a special value used for missing, replace with NaN so it can't be used for merging
name_rating_df["startYear"] = name_rating_df["startYear"].replace('\\N', np.nan)
name_rating_df["startYear"] = name_rating_df["startYear"].astype(float)
#Rename columns for merge
name_rating_df.rename(columns={'primaryTitle': 'title', 'startYear' : 'year', 'titleType': 'type'}, inplace=True)

In [26]:
#Have to take care of duplicate entries for movies in the same year
#ASSUME they are the same movie and aggregate the scores: sum up the numVotes and calculate the
#new average rating taking into account the number of votes

#The ratings need to be weighted to account for the number of votes
name_rating_df["RatingWeight"] = name_rating_df['averageRating'] * name_rating_df['numVotes']
# Group by title and year for duplicates, sum the number of votes and the weighted ratings
name_rating_agg_df = name_rating_df.groupby(['title','year']).agg(
    numVotes=('numVotes', 'sum'),
    RatingWeight = ('RatingWeight','sum'), 
).reset_index()

#Undo the previous weighing, dividing by number of all votes
name_rating_agg_df['averageRating'] = name_rating_agg_df['RatingWeight'] / name_rating_agg_df['numVotes']
#All ratings have 1 space after comma
name_rating_agg_df['averageRating'] = name_rating_agg_df['averageRating'].round(1)

#Drop the temporary weighted ratings
name_rating_agg_df.drop(columns='RatingWeight', inplace=True)

In [27]:
#Get the year of release of the movies from the mixed formatting
dates_as_year = pd.to_datetime(movie_character_oscar_df["release_date"],format='mixed',errors='coerce').dt.year

#Copy of the dataset with the added year column
movie_character_oscar_df['year'] = dates_as_year 

rows_before_ratings = movie_character_oscar_df.shape[0]

# Join movie_character_oscar on IMDB dataset

In [28]:
# Create identifier
imdb_df = name_rating_agg_df.copy(deep=True)
imdb_df['parsed_movie_name'] = imdb_df.apply(lambda x: parse_string(x['title']), axis=1)
imdb_df['release_year'] = imdb_df['year'].fillna(0).astype(int)
imdb_df = imdb_df[imdb_df['parsed_movie_name'].notna()]
imdb_df['movie_identifier'] = imdb_df.apply(lambda x: x['parsed_movie_name'] + '_' + str(x['release_year']), axis=1)

In [29]:
# Drop ratings of movies that share movie identifier
temp = imdb_df.groupby('movie_identifier').agg(count = ('movie_identifier', 'size')).sort_values('count', ascending=False)
temp = temp[temp['count'] == 1]
imdb_df = imdb_df.merge(temp, on='movie_identifier', how='inner')
imdb_df = imdb_df.drop('count', axis=1)
assert imdb_df['movie_identifier'].is_unique

In [30]:
# Select relevant columns
imdb_df = imdb_df[[
    'averageRating',
    'numVotes',
    'movie_identifier'
]]

In [31]:
movie_character_oscar_rating_df = movie_character_oscar_df.merge(imdb_df, on='movie_identifier', how='left')
movie_character_oscar_rating_df['has_rating'] = movie_character_oscar_rating_df['numVotes'].notna()

In [32]:
movie_character_oscar_rating_df['average_rating'] = movie_character_oscar_rating_df['averageRating']
movie_character_oscar_rating_df['number_of_votes'] = movie_character_oscar_rating_df['numVotes']
# Rename columns = movie_character_oscar_rating_df.drop(['numVotes', 'averageRating'], axis=1)

In [33]:
## Stats of dataset
print("\nMerging IMDb dataset, after merge:")
rating_df = movie_character_oscar_rating_df[movie_character_oscar_rating_df['has_rating'] == True]
print('Number of movies with ratings:', rating_df['movie_identifier'].unique().shape[0])
print('Oscar nominated movies with rating:', rating_df[rating_df['oscar_nominated'] == True]['movie_identifier'].unique().shape[0])

# Almost all Oscar nominated movies have rating


Merging IMDb dataset, after merge:
Number of movies with ratings: 36760
Oscar nominated movies with rating: 939


# Rename columns

In [34]:
# Rename columns to a more standardized format
movie_character_oscar_rating_df = movie_character_oscar_rating_df.rename(columns=
                                       {"Actor gender": "actor_gender",
                                        "Actor height (in meters)": "actor_height",
                                        "Actor ethnicity (Freebase ID)":"actor_ethnicity",
                                        "Actor name":"actor_name",
                                        "Actor age at movie release":"actor_age",
                                        })

# Cleaning data

In [37]:
print("Number of rows in data before cleaning: ", len(movie_character_oscar_rating_df))

Number of rows in data before cleaning:  443504


In [50]:
movie_character_oscar_rating_df.isnull().sum() * 100 / len(movie_character_oscar_rating_df)

title                  0.000000
release_date           2.185775
box_office_revenue    77.560744
runtime               11.133158
languages              0.000000
countries              0.000000
genres                 0.000000
movie_identifier       0.000000
actor_gender           9.946697
actor_height          65.646082
actor_ethnicity       76.545646
actor_name             0.000000
actor_age             34.990665
parsed_actor_name      0.000000
actor_identifier       0.000000
identifier             0.000000
category              99.674637
winner                99.674637
oscar_nominated        0.000000
year                   2.189157
has_rating             0.000000
average_rating        33.242316
number_of_votes       33.242316
dtype: float64

In [47]:
data_clean = movie_character_oscar_rating_df.dropna(subset=['title', 'release_date', 'box_office_revenue', 'runtime', 'languages',
       'countries', 'genres', 'movie_identifier', 'actor_gender',
       'actor_height', 'actor_ethnicity', 'actor_name', 'actor_age',
       'parsed_actor_name', 'actor_identifier', 'identifier','year', 'has_rating', 'average_rating',
       'number_of_votes'])
print("Number of rows in data after cleaning: ", len(data_clean))

Number of rows in data after cleaning:  23819


In [52]:
print("\nFINAL STATE OF DATA")
print("Number of rows: ", len(data_clean))
nominated_clean_df = data_clean[data_clean['oscar_nominated'] == True]

print('Number of different Oscar nominated movies in dataset:', 
      nominated_clean_df['movie_identifier'].unique().shape[0],
      "in total", data_clean['movie_identifier'].unique().shape[0], "different movies")

print('Number of different Oscar nominated actors in dataset:', 
      nominated_clean_df['actor_identifier'].unique().shape[0],
      'in total',data_clean['actor_identifier'].unique().shape[0],"different actors")

print('Number of Oscar nominated rows:', nominated_clean_df.shape[0])


FINAL STATE OF DATA
Number of rows:  23819
Number of different Oscar nominated movies in dataset: 394 in total 5989 different movies
Number of different Oscar nominated actors in dataset: 284 in total 2962 different actors
Number of Oscar nominated rows: 519


# Write data

In [53]:
# Finish processed data
path = 'cache/data.csv'
os.makedirs('cache', exist_ok=True)
data_clean.to_csv(path)
print('Processing done, dataset written to',path)

Processing done, dataset written to cache/data.csv
